In [1]:
import sys
from pathlib import Path
import pandas as pd

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

# Verify path
print(f"Python will look in these locations:\n{sys.path}")


# --- Execute the processor ---
import utils
from config import date_str, DOWNLOAD_DIR, DEST_DIR

path_data = f'..\data\{date_str}_df_finviz_merged.parquet'
# path_corr = '..\data\df_corr_emv_matrix.parquet'
# path_cov = '..\data\df_cov_emv_matrix.parquet'
# path_output = f'..\picks\{date_str}_portf.txt'


print(utils.__file__)  # Should point to your src/utils.py
print(f'path_date: {path_data}')
# print(f'path_corr: {path_corr}')
# print(f'path_cov: {path_cov}')  
# print(f'path_output: {path_output}')
# print((f'date_str: {date_str}'))



Python will look in these locations:
['C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\python311.zip', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\DLLs', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9\\Lib', 'C:\\Users\\ping\\.pyenv\\pyenv-win\\versions\\3.11.9', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv', '', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\ping\\Files_win10\\python\\py311\\.venv\\Lib\\site-packages\\Pythonwin', 'c:\\Users\\ping\\Files_win10\\python\\py311\\stocks\\src']


c:\Users\ping\Files_win10\python\py311\stocks\src\utils.py
path_date: ..\data\2025-04-09_df_finviz_merged.parquet


In [2]:
import pandas as pd
# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
# pd.set_option('display.width', None)        # Let the display adjust to the window
# pd.set_option('display.max_colwidth', None) # Show full content of each cell
pd.set_option('display.max_rows', 100)
# pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.4f}'.format)

In [3]:
import pandas as pd

def get_column_values_above_threshold(df, column_name='Avg Volume, M', threshold=0.75):
  """
  Analyzes the number and percentage of values in a DataFrame column that are above a specified threshold,
  and returns the filtered DataFrame.

  Args:
    df (pd.DataFrame): The input DataFrame.
    column_name (str): The name of the column to analyze. Defaults to 'Avg Volume, M'.
    threshold (float): The threshold value to compare against. Defaults to 1.00.

  Returns:
    pd.DataFrame: A DataFrame containing only the rows where the specified column's value is above the threshold.
  """
  
  count_before = len(df)
  above_threshold_df = df[df[column_name] > threshold]
  count_after = len(above_threshold_df)
  percentage = (count_after / len(df)) * 100

  print(f"count_before: {count_before}")
  print(f"count_after above threshold ({threshold}): {count_after}")
  print(f"Percentage above threshold ({threshold}): {percentage:.2f}%")

  return above_threshold_df


In [4]:
df_data = pd.read_parquet(path_data)

# liquidity filter, Avg Volume, M > 0.75M
df_data = get_column_values_above_threshold(df_data, column_name='Avg Volume, M', threshold=0.75)

# Drop specified columns with NaNs in df_data
df_data = df_data.drop(['All-Time High %', 'All-Time Low %', 'Dividend %'], axis=1)

# df_corr = pd.read_parquet(path_corr)
# df_cov = pd.read_parquet(path_cov)

# print(f'\ndf_cov.shape: {df_cov.shape}')
# display(df_cov.head())

# print(f'\ndf_corr.shape: {df_corr.shape}')
# display(df_corr.head())

print(f'\ndf_data.shape: {df_data.shape}')
display(df_data.head())
display((df_data.describe()))

count_before: 1536
count_after above threshold (0.75): 1180
Percentage above threshold (0.75): 76.82%

df_data.shape: (1180, 52)


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",2987140.0000,1.3400,41.8600,-20.5900,5.5579,-11.1800,-9.9600,-17.9000,-10.3000,18.0500,-5.7300,-12.1100,-13.0200,-20.4600,17.5200,-23.5500,21.1900,10.8400,5.4513,10.0300,4.3100,182.7355,58.0600,3.1500,198.8500,15.3300,5.7009,23.1685,3.0640,-0.2582,-0.5617,0.9576,-2.4393,-3.8643,0.6444,-1.0864,-1.7149,0.8095,-2.0674,-3.0125,0.6569,-1.1690,-1.6803,0.7874,-0.7722,-1.0957,0.8485,0.4515,0.6692,1.0940
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2902900.0000,0.9500,53.6900,-7.3600,8.5177,2.1900,2.6400,-7.5500,-4.6500,-8.0300,2.3700,-1.2300,-6.9500,-12.9100,13.2500,-16.6200,13.2500,13.6100,3.4854,6.4100,3.0600,49.1471,24.6100,2.0000,390.4900,10.1300,9.3202,109.8514,10.7863,3.3043,10.7082,1.9930,0.2063,0.4674,1.0421,0.4797,1.0457,1.1083,-0.0243,-0.0457,0.9951,-0.6976,-1.1021,0.8713,-0.4397,-0.6345,0.9175,-0.3937,-0.5451,0.9307
NVDA,NVIDIA Corp,"Technology, Semiconductors",2789650.0000,2.0300,51.7500,-14.8600,21.2279,3.5400,5.1200,-18.4200,-10.4800,31.2100,2.2500,-4.4600,-9.7200,-20.2900,31.9900,-25.3400,51.2200,7.8900,6.9011,12.0400,5.7200,605.6052,295.3600,2.0500,114.3300,18.7200,9.6740,140.0319,13.4750,4.7887,14.2393,2.5363,1.0651,2.2552,1.2425,-0.2324,-0.4410,0.9539,-0.1696,-0.2745,0.9693,-0.4267,-0.5920,0.9260,-0.3448,-0.4790,0.9395,0.6253,0.9027,1.1126
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",2025230.0000,1.2600,47.7300,-12.8900,11.7544,-2.5000,-2.7900,-13.9600,5.7000,3.1900,-0.3700,-8.5100,-4.5000,-21.2000,18.4200,-21.2000,26.0500,10.5400,5.5154,9.7400,4.5600,114.4914,45.3700,2.5200,191.1000,11.9800,7.1652,39.6220,4.5298,4.1575,12.2664,2.1200,-1.1963,-1.8880,0.8133,-0.1758,-0.2762,0.9686,-1.1550,-1.6878,0.8119,-1.1569,-1.6341,0.8097,0.2029,0.2969,1.0365,0.0745,0.1052,1.0131
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1949500.0000,0.9800,47.7700,-15.4300,9.0158,1.3800,-2.9600,-18.1200,-2.0300,3.1500,0.0300,-7.7700,-8.2600,-22.8300,12.9000,-22.8300,12.9000,6.5900,4.0916,6.2800,3.5800,46.0569,21.4000,2.1500,161.0600,9.8800,7.7630,50.3400,5.4846,3.9519,12.5783,2.1647,-0.5033,-0.9088,0.9098,-0.5568,-0.9349,0.9049,-0.8919,-1.3524,0.8587,-1.7120,-2.3013,0.7444,-0.1346,-0.1953,0.9773,0.0450,0.0647,1.0080


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000
mean,59114.5932,0.9948,44.6224,-4.7219,5.3054,-5.3762,-3.3151,-5.4724,-3.9076,5.9045,-3.1519,-6.3433,-5.2951,-16.2327,13.4116,-21.6070,29.3063,4.5577,4.5777,7.7699,3.7529,14.5511,6.5127,2.0863,99.5708,8.3697,-59.7374,577.4473,52.4481,-1.4045,0.1804,1.0603,-1.9763,-2.4727,0.7498,-1.5501,-1.8858,0.7924,-1.0476,-1.3058,0.8549,-0.2696,-0.2572,0.9746,-0.3725,-0.4269,0.9470,0.2279,0.4053,1.0547
std,199351.5250,0.5418,6.2213,14.1438,6.8524,4.6020,7.2523,14.1745,20.3118,46.4956,4.4370,6.7154,12.5791,10.2499,8.8234,13.9210,52.6811,5.8991,2.2897,3.9285,1.9489,39.9149,14.6953,0.8381,114.3941,5.7946,2047.9536,1445.3033,128.7558,4.6412,8.1201,0.8055,2.1133,2.8465,0.3090,1.9285,2.6498,0.3071,1.4729,2.1553,0.2729,1.3560,1.9737,0.2590,0.9814,1.4158,0.1811,0.8132,1.2541,0.1640
min,2880.0000,-3.3900,24.9700,-64.1800,-31.9301,-22.5000,-44.7400,-61.9900,-64.9300,-83.5600,-22.7900,-41.1500,-60.8800,-65.6700,-0.0600,-95.2600,0.2400,0.0200,0.0358,0.0100,0.0100,0.6075,0.7506,0.5900,2.1600,-35.2800,-46856.9203,-15.8745,0.0000,-29.8833,-14.4216,0.0000,-14.0954,-11.2274,0.1114,-11.8165,-10.0702,0.1588,-5.0972,-5.7741,0.3237,-5.0644,-5.6374,0.4040,-3.7345,-4.1288,0.5195,-2.2130,-2.6104,0.6820
25%,8840.0000,0.6800,41.0675,-12.9500,0.5577,-7.7125,-6.8525,-13.7350,-14.0925,-13.1850,-5.2125,-10.3550,-12.1925,-22.0125,8.0000,-29.6525,10.2375,1.1675,3.2183,5.6350,2.6600,2.6491,1.5100,1.5200,34.1500,4.7450,4.2501,13.6799,2.2187,-3.6824,-5.1587,0.5380,-3.0822,-4.1906,0.5674,-2.5704,-3.4483,0.6028,-1.9454,-2.6137,0.6943,-1.1628,-1.5806,0.8009,-0.9981,-1.3370,0.8273,-0.3005,-0.4183,0.9468
50%,17650.0000,0.9850,44.3200,-4.7450,4.5700,-5.0750,-3.2250,-5.1300,-5.0700,0.9950,-2.8200,-6.0300,-5.2400,-14.3750,11.7500,-19.3300,17.1900,2.7000,4.1891,7.0900,3.4500,5.3932,2.6700,1.9200,63.4800,7.9500,7.0394,37.7564,4.3636,-0.5713,-1.0123,0.9051,-1.9518,-2.8570,0.6980,-1.4977,-2.1279,0.7434,-1.2541,-1.7279,0.7911,-0.4799,-0.6768,0.9133,-0.4650,-0.6522,0.9153,0.2089,0.2986,1.0390
75%,45310.0000,1.2400,47.8825,2.0525,8.8329,-2.4400,-0.1375,2.1825,3.3325,15.1650,-0.5900,-1.6925,0.6500,-8.7775,16.8350,-11.1400,31.5275,5.5025,5.7282,9.6700,4.6800,11.8957,5.5900,2.4400,115.1850,11.0275,9.0514,93.4881,9.3286,1.7148,3.9142,1.3741,-0.6898,-1.0754,0.8794,-0.4201,-0.6593,0.9221,-0.2892,-0.3988,0.9485,0.6078,0.8908,1.1215,0.2571,0.3642,1.0490,0.7143,1.0565,1.1401
max,2987140.0000,4.2200,83.4700,78.9300,46.2772,11.9400,41.0200,73.8500,190.7400,963.4400,14.2500,30.5600,57.8000,0.9700,76.7500,0.9700,1113.0700,50.6500,19.9687,41.0300,17.2900,774.5057,295.3600,7.8800,965.1900,54.7900,22703.5218,4653.4302,415.5605,11.7353,64.5641,9.1343,7.7936,21.5756,4.7932,6.4484,17.8111,3.9239,6.5777,14.9903,3.2931,4.5577,9.3492,2.3236,2.7971,8.8243,2.2504,3.6077,7.1822,1.8481


In [5]:
import pandas as pd
import os
import datetime

# --- Configuration ---
STATS_HISTORY_FILE = 'market_daily_summary_stats_history.csv' # Where to store the history

def append_daily_stats(df_daily_data, date_str, filename=STATS_HISTORY_FILE):
    """
    Calculates descriptive stats for df_daily_data, flattens them,
    and appends/updates them as a row in a historical CSV file.

    Args:
        df_daily_data (pd.DataFrame): DataFrame containing the raw data for ALL stocks for ONE specific day.
        date_str (str): The date for this data in 'YYYY-MM-DD' format.
        filename (str): The path to the CSV file storing the historical summary stats.
    """
    print(f"Processing stats for date: {date_str}")

    # 1. Calculate descriptive statistics for the input day's data
    try:
        daily_stats_raw = df_daily_data.describe()
        print("Successfully calculated describe() stats.")
    except Exception as e:
        print(f"Error calculating describe() stats: {e}")
        return # Stop if calculation fails

    # 2. Flatten the describe() output into a single row DataFrame
    try:
        # Unstack turns the stats (mean, std, etc.) into columns alongside the original metrics
        stats_flat = daily_stats_raw.unstack().to_frame().T
        # Create clearer column names like 'Metric_Statistic' (e.g., 'RSI_mean', 'SMA50 %_50%')
        stats_flat.columns = ['_'.join(col).strip() for col in stats_flat.columns.values]
        # Add the date to this row
        stats_flat['Date'] = pd.to_datetime(date_str)
        # Set the Date as the index for this single row
        stats_flat = stats_flat.set_index('Date')
        print("Successfully flattened stats into a single row.")
        # print(stats_flat) # Optional: print the flattened stats for verification

    except Exception as e:
        print(f"Error flattening the stats DataFrame: {e}")
        return # Stop if flattening fails

    # 3. Load existing history, append/update, and save
    try:
        df_summary_history = pd.DataFrame() # Initialize empty
        if os.path.exists(filename):
            print(f"Loading existing history from: {filename}")
            df_summary_history = pd.read_csv(filename, index_col='Date', parse_dates=True)

        # Check if the date already exists - update if it does, append if it doesn't
        current_date = pd.to_datetime(date_str)
        if current_date in df_summary_history.index:
            print(f"Updating existing entry for {date_str}")
            # Update existing row(s) - combines columns, replacing old values for this date
            df_summary_history.update(stats_flat)
            # Alternative: df_summary_history.loc[current_date] = stats_flat.iloc[0] # Direct overwrite
        else:
            print(f"Appending new entry for {date_str}")
            df_summary_history = pd.concat([df_summary_history, stats_flat])

        # Ensure the history is sorted by date
        df_summary_history = df_summary_history.sort_index()

        # Save the updated history back to CSV
        df_summary_history.to_csv(filename, index=True) # index=True saves the Date index
        print(f"Successfully saved updated stats history to: {filename}")

    except Exception as e:
        print(f"Error loading/saving the stats history file '{filename}': {e}")



In [6]:
# Simulate processing for a couple of days
print("-" * 20, "Day 1", "-" * 20)
append_daily_stats(df_data, date_str)

-------------------- Day 1 --------------------
Processing stats for date: 2025-04-09
Successfully calculated describe() stats.
Successfully flattened stats into a single row.
Loading existing history from: market_daily_summary_stats_history.csv


Updating existing entry for 2025-04-09


Successfully saved updated stats history to: market_daily_summary_stats_history.csv


In [7]:
print(f'date_str: {date_str}')
df_data.describe()

date_str: 2025-04-09


,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
count,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000,1180.0000
mean,59114.5932,0.9948,44.6224,-4.7219,5.3054,-5.3762,-3.3151,-5.4724,-3.9076,5.9045,-3.1519,-6.3433,-5.2951,-16.2327,13.4116,-21.6070,29.3063,4.5577,4.5777,7.7699,3.7529,14.5511,6.5127,2.0863,99.5708,8.3697,-59.7374,577.4473,52.4481,-1.4045,0.1804,1.0603,-1.9763,-2.4727,0.7498,-1.5501,-1.8858,0.7924,-1.0476,-1.3058,0.8549,-0.2696,-0.2572,0.9746,-0.3725,-0.4269,0.9470,0.2279,0.4053,1.0547
std,199351.5250,0.5418,6.2213,14.1438,6.8524,4.6020,7.2523,14.1745,20.3118,46.4956,4.4370,6.7154,12.5791,10.2499,8.8234,13.9210,52.6811,5.8991,2.2897,3.9285,1.9489,39.9149,14.6953,0.8381,114.3941,5.7946,2047.9536,1445.3033,128.7558,4.6412,8.1201,0.8055,2.1133,2.8465,0.3090,1.9285,2.6498,0.3071,1.4729,2.1553,0.2729,1.3560,1.9737,0.2590,0.9814,1.4158,0.1811,0.8132,1.2541,0.1640
min,2880.0000,-3.3900,24.9700,-64.1800,-31.9301,-22.5000,-44.7400,-61.9900,-64.9300,-83.5600,-22.7900,-41.1500,-60.8800,-65.6700,-0.0600,-95.2600,0.2400,0.0200,0.0358,0.0100,0.0100,0.6075,0.7506,0.5900,2.1600,-35.2800,-46856.9203,-15.8745,0.0000,-29.8833,-14.4216,0.0000,-14.0954,-11.2274,0.1114,-11.8165,-10.0702,0.1588,-5.0972,-5.7741,0.3237,-5.0644,-5.6374,0.4040,-3.7345,-4.1288,0.5195,-2.2130,-2.6104,0.6820
25%,8840.0000,0.6800,41.0675,-12.9500,0.5577,-7.7125,-6.8525,-13.7350,-14.0925,-13.1850,-5.2125,-10.3550,-12.1925,-22.0125,8.0000,-29.6525,10.2375,1.1675,3.2183,5.6350,2.6600,2.6491,1.5100,1.5200,34.1500,4.7450,4.2501,13.6799,2.2187,-3.6824,-5.1587,0.5380,-3.0822,-4.1906,0.5674,-2.5704,-3.4483,0.6028,-1.9454,-2.6137,0.6943,-1.1628,-1.5806,0.8009,-0.9981,-1.3370,0.8273,-0.3005,-0.4183,0.9468
50%,17650.0000,0.9850,44.3200,-4.7450,4.5700,-5.0750,-3.2250,-5.1300,-5.0700,0.9950,-2.8200,-6.0300,-5.2400,-14.3750,11.7500,-19.3300,17.1900,2.7000,4.1891,7.0900,3.4500,5.3932,2.6700,1.9200,63.4800,7.9500,7.0394,37.7564,4.3636,-0.5713,-1.0123,0.9051,-1.9518,-2.8570,0.6980,-1.4977,-2.1279,0.7434,-1.2541,-1.7279,0.7911,-0.4799,-0.6768,0.9133,-0.4650,-0.6522,0.9153,0.2089,0.2986,1.0390
75%,45310.0000,1.2400,47.8825,2.0525,8.8329,-2.4400,-0.1375,2.1825,3.3325,15.1650,-0.5900,-1.6925,0.6500,-8.7775,16.8350,-11.1400,31.5275,5.5025,5.7282,9.6700,4.6800,11.8957,5.5900,2.4400,115.1850,11.0275,9.0514,93.4881,9.3286,1.7148,3.9142,1.3741,-0.6898,-1.0754,0.8794,-0.4201,-0.6593,0.9221,-0.2892,-0.3988,0.9485,0.6078,0.8908,1.1215,0.2571,0.3642,1.0490,0.7143,1.0565,1.1401
max,2987140.0000,4.2200,83.4700,78.9300,46.2772,11.9400,41.0200,73.8500,190.7400,963.4400,14.2500,30.5600,57.8000,0.9700,76.7500,0.9700,1113.0700,50.6500,19.9687,41.0300,17.2900,774.5057,295.3600,7.8800,965.1900,54.7900,22703.5218,4653.4302,415.5605,11.7353,64.5641,9.1343,7.7936,21.5756,4.7932,6.4484,17.8111,3.9239,6.5777,14.9903,3.2931,4.5577,9.3492,2.3236,2.7971,8.8243,2.2504,3.6077,7.1822,1.8481


In [8]:
print(f'date_str: {date_str}')
df_data.head()

date_str: 2025-04-09


,Company,Info,"MktCap AUM, M",Beta,RSI,Perf YTD %,Perf 3D %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,ATR,ATR/Price %,Volatility W %,Volatility M %,"Volume, M","Avg Volume, M",Rel Volume,Price,Change %,Sharpe 3d,Sortino 3d,Omega 3d,Sharpe 5d,Sortino 5d,Omega 5d,Sharpe 10d,Sortino 10d,Omega 10d,Sharpe 15d,Sortino 15d,Omega 15d,Sharpe 30d,Sortino 30d,Omega 30d,Sharpe 60d,Sortino 60d,Omega 60d,Sharpe 120d,Sortino 120d,Omega 120d,Sharpe 250d,Sortino 250d,Omega 250d
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AAPL,Apple Inc,"Technology, Consumer Electronics",2987140.0000,1.3400,41.8600,-20.5900,5.5579,-11.1800,-9.9600,-17.9000,-10.3000,18.0500,-5.7300,-12.1100,-13.0200,-20.4600,17.5200,-23.5500,21.1900,10.8400,5.4513,10.0300,4.3100,182.7355,58.0600,3.1500,198.8500,15.3300,5.7009,23.1685,3.0640,-0.2582,-0.5617,0.9576,-2.4393,-3.8643,0.6444,-1.0864,-1.7149,0.8095,-2.0674,-3.0125,0.6569,-1.1690,-1.6803,0.7874,-0.7722,-1.0957,0.8485,0.4515,0.6692,1.0940
MSFT,Microsoft Corporation,"Technology, Software - Infrastructure",2902900.0000,0.9500,53.6900,-7.3600,8.5177,2.1900,2.6400,-7.5500,-4.6500,-8.0300,2.3700,-1.2300,-6.9500,-12.9100,13.2500,-16.6200,13.2500,13.6100,3.4854,6.4100,3.0600,49.1471,24.6100,2.0000,390.4900,10.1300,9.3202,109.8514,10.7863,3.3043,10.7082,1.9930,0.2063,0.4674,1.0421,0.4797,1.0457,1.1083,-0.0243,-0.0457,0.9951,-0.6976,-1.1021,0.8713,-0.4397,-0.6345,0.9175,-0.3937,-0.5451,0.9307
NVDA,NVIDIA Corp,"Technology, Semiconductors",2789650.0000,2.0300,51.7500,-14.8600,21.2279,3.5400,5.1200,-18.4200,-10.4800,31.2100,2.2500,-4.4600,-9.7200,-20.2900,31.9900,-25.3400,51.2200,7.8900,6.9011,12.0400,5.7200,605.6052,295.3600,2.0500,114.3300,18.7200,9.6740,140.0319,13.4750,4.7887,14.2393,2.5363,1.0651,2.2552,1.2425,-0.2324,-0.4410,0.9539,-0.1696,-0.2745,0.9693,-0.4267,-0.5920,0.9260,-0.3448,-0.4790,0.9395,0.6253,0.9027,1.1126
AMZN,Amazon.com Inc,"Consumer Cyclical, Internet Retail",2025230.0000,1.2600,47.7300,-12.8900,11.7544,-2.5000,-2.7900,-13.9600,5.7000,3.1900,-0.3700,-8.5100,-4.5000,-21.2000,18.4200,-21.2000,26.0500,10.5400,5.5154,9.7400,4.5600,114.4914,45.3700,2.5200,191.1000,11.9800,7.1652,39.6220,4.5298,4.1575,12.2664,2.1200,-1.1963,-1.8880,0.8133,-0.1758,-0.2762,0.9686,-1.1550,-1.6878,0.8119,-1.1569,-1.6341,0.8097,0.2029,0.2969,1.0365,0.0745,0.1052,1.0131
GOOG,Alphabet Inc,"Communication Services, Internet Content & Inf...",1949500.0000,0.9800,47.7700,-15.4300,9.0158,1.3800,-2.9600,-18.1200,-2.0300,3.1500,0.0300,-7.7700,-8.2600,-22.8300,12.9000,-22.8300,12.9000,6.5900,4.0916,6.2800,3.5800,46.0569,21.4000,2.1500,161.0600,9.8800,7.7630,50.3400,5.4846,3.9519,12.5783,2.1647,-0.5033,-0.9088,0.9098,-0.5568,-0.9349,0.9049,-0.8919,-1.3524,0.8587,-1.7120,-2.3013,0.7444,-0.1346,-0.1953,0.9773,0.0450,0.0647,1.0080
